$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu \frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2}

\frac{\partial v}{\partial t}+u\\frac{\\partial v}{\\partial x}+v\\frac{\\partial v}{\\partial y} = -\\frac{1}{\\rho}\\frac{\\partial p}{\\partial y}+\\nu\\left(\\frac{\\partial^2 v}{\\partial x^2}+\\frac{\\partial^2 v}{\\partial y^2}\\right)

\\frac{\\partial^2 p}{\\partial x^2}+\\frac{\\partial^2 p}{\\partial y^2} = -\\rho\\left(\\frac{\\partial u}{\\partial x}\\frac{\\partial u}{\\partial x}+2\\frac{\\partial u}{\\partial y}\\frac{\\partial v}{\\partial x}+\\frac{\\partial v}{\\partial y}\\frac{\\partial v}{\\partial y} \\right)$


Now we will import the packages we need:

In [1]:
using MethodOfLines, DomainSets, ModelingToolkit, OrdinaryDiffEq


Now we'll construct the system:

In [5]:
@parameters t, x, y
@variables p(..), u(..), v(..)

Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

ρ = 1.0
ν = 0.1

eqs = [Dt(u(t,x,y)) + u(t,x,y)*Dx(u(t,x,y)) + v(t,x,y)*Dy(v(t,x,y)) ~  -(1/ρ)*Dx(p(t,x,y)) + ν*(Dxx(u(t,x,y)) + Dyy(u(t,x,y))),
       Dt(v(t,x,y)) + u(t,x,y)*Dx(v(t,x,y)) + v(t,x,y)*Dy(v(t,x,y)) ~  -(1/ρ)*Dy(p(t,x,y)) + ν*(Dxx(v(t,x,y)) + Dyy(v(t,x,y))),
       Dxx(p(t,x,y)) + Dyy(p(t,x,y)) ~  -ρ*((Dx(u(t,x,y)))^2 + 2*Dx(u(t,x,y))*Dy(v(t,x,y)) + (Dy(v(t,x,y)))^2)]

bcs = [u(0,x,y) ~ 0.0,
       v(0,x,y) ~ 0.0,
       u(t,x,0) ~ 0.0,
       u(t,x,2) ~ 1.0,
       v(t,x,0) ~ 0.0,
       v(t,x,2) ~ 0.0,
       u(t,0,y) ~ 0.0,
       u(t,2,y) ~ 0.0,
       v(t,0,y) ~ 0.0,
       v(t,2,y) ~ 0.0,
       Dy(p(t,x,0)) ~ 0.0,
       p(t,x,2) ~ 0.0,
       Dx(p(t,0,y)) ~ 0.0,
       Dx(p(t,2,y)) ~ 0.0]

domains = [t ∈ IntervalDomain(0.0,0.1),
           x ∈ IntervalDomain(0.0,2.0),
           y ∈ IntervalDomain(0.0,2.0)]

@named pdesys = PDESystem(eqs, bcs, domains, [t,x,y],[u(t,x,y),v(t,x,y),p(t,x,y)])

PDESystem
Equations: Equation[u(t, x, y)*Differential(x)(u(t, x, y)) + v(t, x, y)*Differential(y)(v(t, x, y)) + Differential(t)(u(t, x, y)) ~ 0.1(Differential(x)(Differential(x)(u(t, x, y))) + Differential(y)(Differential(y)(u(t, x, y)))) - Differential(x)(p(t, x, y)), u(t, x, y)*Differential(x)(v(t, x, y)) + v(t, x, y)*Differential(y)(v(t, x, y)) + Differential(t)(v(t, x, y)) ~ 0.1(Differential(x)(Differential(x)(v(t, x, y))) + Differential(y)(Differential(y)(v(t, x, y)))) - Differential(y)(p(t, x, y)), Differential(x)(Differential(x)(p(t, x, y))) + Differential(y)(Differential(y)(p(t, x, y))) ~ -(Differential(x)(u(t, x, y))^2) - (Differential(y)(v(t, x, y))^2) - 2.0Differential(x)(u(t, x, y))*Differential(y)(v(t, x, y))]
Boundary Conditions: Equation[u(0, x, y) ~ 0.0, v(0, x, y) ~ 0.0, u(t, x, 0) ~ 0.0, u(t, x, 2) ~ 1.0, v(t, x, 0) ~ 0.0, v(t, x, 2) ~ 0.0, u(t, 0, y) ~ 0.0, u(t, 2, y) ~ 0.0, v(t, 0, y) ~ 0.0, v(t, 2, y) ~ 0.0, Differential(y)(p(t, x, 0)) ~ 0.0, p(t, x, 2) ~ 0.0, Diff

Let's discretize:

In [6]:
disc = MOLFiniteDifference([x => 41, y => 41], t, advection_scheme=WENOScheme())

prob = discretize(pdesys, disc)

┌ Warning: The number of initial conditions must match the number of dependent variables.
└ @ MethodOfLines /Users/alexjones/.julia/dev/MethodOfLines.jl/src/discretization/generate_ic_defaults.jl:4


The system of equations is:


Equation[

20.0((0.049999999999999996(2(p

(t))[3, 3] + 5(p(t))[4, 3] - (p(t))[5, 3])) / ((0.3 / ((1.0e-6 + (13//12)*(((p(t))[3, 3] + (p(t))[5, 3] - 2(p(t))[4, 3])^2) + (1//4)*((3(p(t))[3, 3] + (p(t))[5, 3] - 4(p(t))[4, 3])^2))^2) + 0.1 / ((1.0e-6 + (1//4)*((3(p(t))[3, 3] + (p(t))[1, 3] - 4(p(t))[2, 3])^2) + (13//12)*(((p(t))[1, 3] + (p(t))[3, 3] - 2(p(t))[2, 3])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((p(t))[2, 3] - (p(t))[4, 3])^2) + (13//12)*(((p(t))[2, 3] + (p(t))[4, 3] - 2(p(t))[3, 3])^2))^2))*((1.0e-6 + (13//12)*(((p(t))[3, 3] + (p(t))[5, 3] - 2(p(t))[4, 3])^2) + (1//4)*((3(p(t))[3, 3] + (p(t))[5, 3] - 4(p(t))[4, 3])^2))^2)) + (-0.049999999999999996(5(p(t))[2, 3] + 2(p(t))[3, 3] - (p(t))[1, 3])) / ((0.1 / ((1.0e-6 + (13//12)*(((p(t))[3, 3] + (p(t))[5, 3] - 2(p(t))[4, 3])^2) + (1//4)*((3(p(t))[3, 3] + (p(t))[5, 3] - 4(p(t))[4, 3])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((p(t))[2, 3] - (p(t))[4, 3])^2) + (13//12)*(((p(t))[2, 3] + (p(t))[4, 3] - 2(p(t))[3, 3])^2))^2) + 0.3 / ((1.0e-6 + (1//4)*((3(p(t))[3, 3] + (p(t))[1, 3] - 4(p(t))[2

...

v(t))[9, 5] - 2(v(t))[9, 4])^2))^2) + 0.3 / ((1.0e-6 + (13//12)*(((v(t))[9, 2] + (v(t))[9, 4] - 2(v(t))[9, 3])^2) + (1//4)*((3(v(t))[9, 4] + (v(t))[9, 2] - 4(v(t))[9, 3])^2))^2) + 0.1 / ((1.0e-6 + (13//12)*(((v(t))[9, 4] + (v(t))[9, 6] - 2(v(t))[9, 5])^2) + (1//4)*((3(v(t))[9, 4] + (v(t))[9, 6] - 4(v(t))[9, 5])^2))^2))*((1.0e-6 + (13//12)*(((v(t))[9, 2] + (v(t))[9, 4] - 2(v(t))[9, 3])^2) + (1//4)*((3(v(t))[9, 4] + (v(t))[9, 2] - 4(v(t))[9, 3])^2))^2)) + (0.09999999999999999(2(v(t))[9, 5] + 5(v(t))[9, 4] - (v(t))[9, 3])) / ((0.1 / ((1.0e-6 + (13//12)*(((v(t))[9, 2] + (v(t))[9, 4] - 2(v(t))[9, 3])^2) + (1//4)*((3(v(t))[9, 4] + (v(t))[9, 2] - 4(v(t))[9, 3])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((v(t))[9, 3] - (v(t))[9, 5])^2) + (13//12)*(((v(t))[9, 3] + (v(t))[9, 5] - 2(v(t))[9, 4])^2))^2) + 0.3 / ((1.0e-6 + (13//12)*(((v(t))[9, 4] + (v(t))[9, 6] - 2(v(t))[9, 5])^2) + (1//4)*((3(v(t))[9, 4] + (v(t))[9, 6] - 4(v(t))[9, 5])^2))^2))*((1.0e-6 + (1//4)*(((v(t))[9, 3] - (v(t))[9, 5])^2) + (13//12)

] - (u(t))[11, 4])) / ((0.1 / ((1.0e-6 + (13//12)*(((u(t))[10, 4] + (u(t))[12, 4] - 2(u(t))[11, 4])^2) + (1//4)*((3(u(t))[10, 4] + (u(t))[12, 4] - 4(u(t))[11, 4])^2))^2) + 0.3 / ((1.0e-6 + (1//4)*((3(u(t))[10, 4] + (u(t))[8, 4] - 4(u(t))[9, 4])^2) + (13//12)*(((u(t))[8, 4] + (u(t))[10, 4] - 2(u(t))[9, 4])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((u(t))[9, 4] - (u(t))[11, 4])^2) + (13//12)*(((u(t))[9, 4] + (u(t))[11, 4] - 2(u(t))[10, 4])^2))^2))*((1.0e-6 + (1//4)*(((u(t))[9, 4] - (u(t))[11, 4])^2) + (13//12)*(((u(t))[9, 4] + (u(t))[11, 4] - 2(u(t))[10, 4])^2))^2)) + (0.016666666666666666(2(u(t))[8, 4] + 11(u(t))[10, 4] - 7(u(t))[9, 4])) / ((0.3 / ((1.0e-6 + (13//12)*(((u(t))[10, 4] + (u(t))[12, 4] - 2(u(t))[11, 4])^2) + (1//4)*((3(u(t))[10, 4] + (u(t))[12, 4] - 4(u(t))[11, 4])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((u(t))[9, 4] - (u(t))[11, 4])^2) + (13//12)*(((u(t))[9, 4] + (u(t))[11, 4] - 2(u(t))[10, 4])^2))^2) + 0.1 / ((1.0e-6 + (1//4)*((3(u(t))[10, 4] + (u(t))[8, 4] - 4(u(t))[9, 4])^2) + (13/

Excessive output truncated after 524289 bytes.

] + (p(t))[11, 4] - 2(p(t))[10, 4])^2) + (1//4)*((3(p(t))[11, 4] + (p(t))[9, 4] - 4(p(t))[10, 4])^2))^2)) + (0.049999999999999996(2(p(t))[11, 4] + 5(p(t))[12, 4] - (p(t))[13, 4])) / ((0.1 / ((1.0e-6 + (13//12)*(((p(t))[9, 4] + (p(t))[11, 4] - 2(p(t))[10, 4])^2) + (1//4)*((3(p(t))[11, 4] + (p(t))[9, 4] - 4(p(t))[10, 4])^2))^2) + 0.6 / ((1.0e-6 + (1//4)*(((p(t))[10, 4] - (p(t))[12, 4])^2

ArgumentError: ArgumentError: SymbolicUtils.BasicSymbolic{Real}[(p(t))[1, 3], (p(t))[41, 3], (p(t))[1, 4], (p(t))[5, 4], (p(t))[7, 4], (p(t))[8, 4], (p(t))[10, 4], (p(t))[19, 4], (p(t))[22, 4], (p(t))[23, 4], (p(t))[24, 4], (p(t))[25, 4], (p(t))[27, 4], (p(t))[28, 4], (p(t))[29, 4], (p(t))[33, 4], (p(t))[34, 4], (p(t))[38, 4], (p(t))[40, 4], (p(t))[41, 4], (p(t))[3, 5], (p(t))[4, 5], (p(t))[5, 5], (p(t))[1, 5], (p(t))[8, 5], (p(t))[9, 5], (p(t))[10, 5], (p(t))[21, 5], (p(t))[22, 5], (p(t))[26, 5], (p(t))[27, 5], (p(t))[29, 5], (p(t))[34, 5], (p(t))[35, 5], (p(t))[36, 5], (p(t))[38, 5], (p(t))[41, 5], (p(t))[4, 6], (p(t))[1, 6], (p(t))[9, 6], (p(t))[12, 6], (p(t))[13, 6], (p(t))[14, 6], (p(t))[20, 6], (p(t))[21, 6], (p(t))[26, 6], (p(t))[31, 6], (p(t))[35, 6], (p(t))[38, 6], (p(t))[41, 6], (p(t))[13, 7], (p(t))[15, 7], (p(t))[20, 7], (p(t))[32, 7], (p(t))[33, 7], (p(t))[34, 7], (p(t))[40, 7], (p(t))[41, 7], (p(t))[10, 8], (p(t))[14, 8], (p(t))[17, 8], (p(t))[19, 8], (p(t))[20, 8], (p(t)

And Finally Solve

In [4]:
sol = solve(prob, QRBDF(), saveat=0.001)


UndefVarError: UndefVarError: `QRBDF` not defined